In [54]:
# Imports

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
plt.style.use("ggplot")
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF
import keras.losses.crf_loss

ModuleNotFoundError: No module named 'keras.losses.crf_loss'; 'keras.losses' is not a package

In [29]:
def read_data():
    os.chdir('D:/TU_Graz/Thesis/Datasets/Reddit_features')

    train = pd.read_csv("train_final_all.csv")
    test = pd.read_csv("test_final_all.csv")
    data = train.append(test)
    #data = data[["Sentence", "Token", "POS_tag", "BIO"]]
    return train, test, data

In [30]:
# Read the data
print("Loading the data...")
train, test, data = read_data()
print("Done.")

Loading the data...
Done.


In [31]:
words = list(set(data["Token"].values))
words.append("ENDPAD")
n_words = len(words); n_words

9743

In [32]:
tags = list(set(data["BIO"].values))
n_tags = len(tags); n_tags

17

In [33]:
class SentenceGetter(object):
    
    def __init__(self, data, category):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, p, t) for w, p, t in zip(s["Token"].values.tolist(),
                                                           s["POS_tag"].values.tolist(),
                                                           s[category].values.tolist())]
        self.grouped = self.data.groupby("Sent_id").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped
            self.n_sent += 1
            return s
        except:
            return None

In [34]:
# Create list of sentences
print("Creating sentence list...")
getter = SentenceGetter(data, "BIO")
sents = getter.sentences
print("Done.")

Creating sentence list...
Done.


In [35]:
def remove_sents_over_threshold(sents, threshold):
    sentences = list()
    for s in sents:
        if len(s) < threshold:
            sentences.append(s)
    return sentences

In [36]:
# Remove submissions longer than a certain threshold
print("Removing submissions longer than threshold...")
sentences = remove_sents_over_threshold(sents, 200)
print("Done.")

Removing submissions longer than threshold...
Done.


In [37]:
max_len = 200
word2idx = {w: i + 1 for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate(tags)}

In [38]:
from keras.preprocessing.sequence import pad_sequences
X = [[word2idx[w[0]] for w in s] for s in sentences]

In [39]:
X = pad_sequences(maxlen=max_len, sequences=X, padding="post", value=0)

In [40]:
y = [[tag2idx[w[2]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding="post", value=tag2idx["O"])
y = [to_categorical(i, num_classes=n_tags) for i in y]

In [41]:
X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size=0.1)

In [42]:
input = Input(shape=(max_len,))
model = Embedding(input_dim=n_words + 1, output_dim=20,
                  input_length=max_len, mask_zero=True)(input)  # 20-dim embedding
model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model)  # variational biLSTM
model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags)  # CRF layer
out = crf(model)  # output

In [43]:
model = Model(input, out)

In [53]:
model.compile(optimizer="rmsprop", loss=keras.losses.crf_loss, metrics=[crf.accuracy])

AttributeError: module 'keras.losses' has no attribute 'crf_loss'

In [45]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 200, 20)           194880    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200, 100)          28400     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 200, 50)           5050      
_________________________________________________________________
crf_1 (CRF)                  (None, 200, 17)           1190      
Total params: 229,520
Trainable params: 229,520
Non-trainable params: 0
_________________________________________________________________


In [46]:
history = model.fit(X_tr, np.array(y_tr), batch_size=32, epochs=5,
                    validation_split=0.1, verbose=1)

Epoch 1/5


AttributeError: in user code:

    C:\Users\Kiki\anaconda3\lib\site-packages\keras\engine\training.py:853 train_function  *
        return step_function(self, iterator)
    C:\Users\Kiki\anaconda3\lib\site-packages\keras_contrib\losses\crf_losses.py:54 crf_loss  *
        crf, idx = y_pred._keras_history[:2]
    C:\Users\Kiki\anaconda3\lib\site-packages\tensorflow\python\framework\ops.py:401 __getattr__
        self.__getattribute__(name)

    AttributeError: 'Tensor' object has no attribute '_keras_history'
